In [4]:
# !pip install beautifulsoup4
# !pip install wikipedia-api

In [7]:
import requests, json
from bs4 import BeautifulSoup
import wikipediaapi

def food_info(name):
    url = f"https://www.10000recipe.com/recipe/list.html?q={name}" # 만개의 레시피
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
    else :
        print("HTTP response error :", response.status_code)
        return

    food_list = soup.find_all(attrs={'class':'common_sp_link'})
    food_id = food_list[0]['href'].split('/')[-1]
    new_url = f'https://www.10000recipe.com/recipe/{food_id}'
    new_response = requests.get(new_url)
    if new_response.status_code == 200:
        html = new_response.text
        soup = BeautifulSoup(html, 'html.parser')
    else :
        print("HTTP response error :", response.status_code)
        return

    food_info = soup.find(attrs={'type':'application/ld+json'})
    result = json.loads(food_info.text)
    ingredient = ','.join(result['recipeIngredient'])
    wiki_ko = wikipediaapi.Wikipedia('KFR ()', 'ko')
    page_py_ko = wiki_ko.page(name)

    res = {
        '음식': name,
        '정보': page_py_ko.summary,
        '재료': ingredient
    }

    return res

In [45]:
# food_info('김치찌개')['정보']

In [10]:
import pandas as pd

In [11]:
# 40개의 음식 클래스 선정
food_names = ['간장게장', '갈비찜', '갈비탕', '감자전', '감자탕', '곱창구이', '김밥', '김치전', '김치찌개',\
              '닭갈비', '닭볶음탕', '도토리묵', '된장찌개', '떡볶이', '막국수', '물냉면', '물회', '미역국',\
              '배추김치', '불고기', '비빔냉면', '비빔밥', '삼겹살', '삼계탕', '설렁탕', '순대', '순두부찌개',\
              '양념게장', '양념치킨', '육회', '잡채', '제육볶음', '족발', '주꾸미볶음', '짜장면', '칼국수',\
              '파전', '해물찜', '황태구이', '후라이드치킨']

In [13]:
rows = []
for name in food_names :
    a = food_info(name)['음식']
    b = food_info(name)['정보']
    c = food_info(name)['재료']
    row = [a, b, c]
    rows.append(row)

In [14]:
food_info = pd.DataFrame(rows, columns = ['food', 'info', 'ingredient'])
# food_info.head()

,food,info,ingredient
0,간장게장,"게장(-醬)은 한국의 게 요리이다. 게를 간장물에 담가 숙성해 만든 젓갈의 하나로,...","꽃게 5마리,간장 3머그컵,물 3.5머그컵,청양고추 4개,홍고추 2개,사과 1개,생..."
1,갈비찜,갈비찜은 갈비에 양념과 간을 하여 국물을 붓고 쪄서 만든 찜류의 음식이다. 과거엔 ...,"돼지갈비 2kg,감자,양파,대파,당근,버섯,밤,청양고추,진간장 2컵,설탕 1컵,물 ..."
2,갈비탕,"갈비탕(영어: galbi-tang, short rib soup)은 소의 갈비부위를 ...","갈비탕용 소고기 1kg,무 1토막,대파 흰부분 5대,양파 1/2개,당면 1줌,물 1..."
3,감자전,"감자전은 감자를 갈아서 기름에 부친 전으로, 강원도 지방의 향토음식이다.","감자 3개,물 감자가잠길만큼,소금 1꼬집,식용유 넉넉히,간장 3큰술,식초 1/2큰술..."
4,감자탕,"감자탕(Gamja-tang, pork back-bone stew)은 돼지 등뼈와 감...","감자탕용 돼지등뼈 1.5kg,감자 작은거 6개,데친 얼갈이 배추 200g,깻잎 10..."


In [26]:
cols = ['food', 'info', 'ingredient']
data = []

In [27]:
for idx in food_info.index : 
    tuple_t = []
    tmp = food_info.loc[idx]
    tuple_t.append(str(tmp.food))
    tuple_t.append(str(tmp.info))
    tuple_t.append(str(tmp.ingredient))
    data.append(dict(zip(cols,tuple_t)))

In [28]:
data

[{'food': '간장게장',
  'info': '게장(-醬)은 한국의 게 요리이다. 게를 간장물에 담가 숙성해 만든 젓갈의 하나로, 보통 쌀밥과 먹는다. 게젓, 장해(醬蟹), 해서(蟹胥), 해해(蟹醢)라고도 한다. 게장이라는 음식의 기원 자체가 간장으로 절인 것이지만, 고춧가루를 이용한 양념게장과의 구분을 위해서 오늘날에는 간장게장(-醬-醬)이라는 용어가 사용된다. 양념게장은 한국전쟁이 끝난 후 등장하기 시작했다. 게장은 경기도, 경상도, 전라도, 제주도 등 각각 지역별로 독특한 형태로 나타난다.',
  'ingredient': '꽃게 5마리,간장 3머그컵,물 3.5머그컵,청양고추 4개,홍고추 2개,사과 1개,생강가루 1큰술,소주or청주 3큰술,통마늘 7~10개'},
 {'food': '갈비찜',
  'info': "갈비찜은 갈비에 양념과 간을 하여 국물을 붓고 쪄서 만든 찜류의 음식이다. 과거엔 갈비라고 하면 일반적인 늑골을 의미했으며, 식용으로 사용하는 소의 늑골을 따로 분류하여 '가리'라고 불렀다. 이에 따라 소의 늑골로 만든 갈비찜을 가리찜이라고도 하였으나 1989년 한글 맞춤법이 수정됨에 따라 가리는 비표준어가 되었으며, 갈비라는 용어로 통일되었다. 갈비찜은 일반적으로 소와 돼지의 갈비로 만든다. 돼지의 갈비로 만든 갈비찜은 돼지갈비찜이라고 하며, 소의 갈비로 만들었으면 소갈비찜이라고 부른다.\n갈비찜의 재료로 선호하는 것은 송아지의 갈비인데 왜냐하면 송아지의 갈비가 더 부드럽고 맛있기 때문이다. 한국에서 갈비는 소의 다른 어떤 부위보다 비싸며, 따라서 갈비찜은 고가의 음식에 속한다. 주로 잔칫상이나 명절에 많이 먹는다.[1]갈비는 매우 지방이 많은 조직이므로 조리할 때 지방을 적절하게 제거해 주어야 한다. 조리하기 전 가장 먼저 해야할 일은 갈비를 물에 한 두시간 담가 핏물을 빼는 일이다.\n\n한국 요리",
  'ingredient': '돼지갈비 2kg,감자,양파,대파,당근,버섯,밤,청양고추,진간장 2컵,설탕 1컵,물 2컵,맛술 1컵,파 1컵,

In [29]:
# json에 포함시킬 meta data 구성
res = {
        'meta':{
            'desc':'음식 재료, 정보',
            'cols':{
                '음식':'food'
                ,'정보':'info'
                ,'재료':'ingredient' 
                  },
        },
        'data':data
}
res

{'meta': {'desc': '음식 재료, 정보',
  'cols': {'음식': 'food', '정보': 'info', '재료': 'ingredient'}},
 'data': [{'food': '간장게장',
   'info': '게장(-醬)은 한국의 게 요리이다. 게를 간장물에 담가 숙성해 만든 젓갈의 하나로, 보통 쌀밥과 먹는다. 게젓, 장해(醬蟹), 해서(蟹胥), 해해(蟹醢)라고도 한다. 게장이라는 음식의 기원 자체가 간장으로 절인 것이지만, 고춧가루를 이용한 양념게장과의 구분을 위해서 오늘날에는 간장게장(-醬-醬)이라는 용어가 사용된다. 양념게장은 한국전쟁이 끝난 후 등장하기 시작했다. 게장은 경기도, 경상도, 전라도, 제주도 등 각각 지역별로 독특한 형태로 나타난다.',
   'ingredient': '꽃게 5마리,간장 3머그컵,물 3.5머그컵,청양고추 4개,홍고추 2개,사과 1개,생강가루 1큰술,소주or청주 3큰술,통마늘 7~10개'},
  {'food': '갈비찜',
   'info': "갈비찜은 갈비에 양념과 간을 하여 국물을 붓고 쪄서 만든 찜류의 음식이다. 과거엔 갈비라고 하면 일반적인 늑골을 의미했으며, 식용으로 사용하는 소의 늑골을 따로 분류하여 '가리'라고 불렀다. 이에 따라 소의 늑골로 만든 갈비찜을 가리찜이라고도 하였으나 1989년 한글 맞춤법이 수정됨에 따라 가리는 비표준어가 되었으며, 갈비라는 용어로 통일되었다. 갈비찜은 일반적으로 소와 돼지의 갈비로 만든다. 돼지의 갈비로 만든 갈비찜은 돼지갈비찜이라고 하며, 소의 갈비로 만들었으면 소갈비찜이라고 부른다.\n갈비찜의 재료로 선호하는 것은 송아지의 갈비인데 왜냐하면 송아지의 갈비가 더 부드럽고 맛있기 때문이다. 한국에서 갈비는 소의 다른 어떤 부위보다 비싸며, 따라서 갈비찜은 고가의 음식에 속한다. 주로 잔칫상이나 명절에 많이 먹는다.[1]갈비는 매우 지방이 많은 조직이므로 조리할 때 지방을 적절하게 제거해 주어야 한다. 조리하기 전 가장 먼저 해야할 일은 갈비를 물에 한 두시

In [30]:
from hdfs import InsecureClient # hadoop 서버와 연결하는  client객체 모듈(hdfs 접근)
import requests # web 사용
import json # json 기능 사용
import datetime as dt

In [31]:
## hdfs client 연결객체 생성
client=InsecureClient('http://localhost:9870',user='lab11')
client

<InsecureClient(url='http://localhost:9870')>

In [33]:
file_dir='/data/'
file_name='food_info'+'.json'
client.write(file_dir+file_name,json.dumps(res, ensure_ascii=False),encoding='utf=8')

In [34]:
file_name = '/data/food_info'+'.json'

file_name
food_info = spark.read.json(file_name, encoding='UTF-8') # spark.dataframe

In [35]:
food_info.show()

+-----------------------------+--------------------+
|                         data|                meta|
+-----------------------------+--------------------+
|[{간장게장, 게장(-醬)은 한...|{{food, ingredien...|
+-----------------------------+--------------------+



In [45]:
food_info[['meta']]

DataFrame[meta: struct<cols:struct<음식:string,재료:string,정보:string>,desc:string>]